### Preliminaries

In [ ]:
%matplotlib inline

# IMPORT MODULES
import sys
import os
import time
import warnings
import platform
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt 
import sklearn
from scipy.stats import norm
import seaborn as sns 
import hyperopt
import scipy
import xgboost

# Print Anaconda and Python versions
print(f"Anaconda Version is {sys.version}")
print(f"Python version is {platform.python_version()}")

# Print versions of imported libraries
print(f"Pandas version is: {pd.__version__}")
print(f"Numpy version is: {np.__version__}")
print(f"Matplotlib version is: {matplotlib.__version__}")
print(f"Sklearn version is: {sklearn.__version__}")
print(f"Seaborn version is: {sns.__version__}")
print(f"Hyperopt version is: {hyperopt.__version__}")
print(f"Scipy version is: {scipy.__version__}")
print(f"Xgboost version is: {xgboost.__version__}")

# Set seaborn settings
sns.set(context='paper', palette='winter_r', style='darkgrid', rc={'figure.facecolor': 'gray'}, font_scale=1.5)

# Filter out warnings (deprecations, etc.)
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses


In [ ]:
#Alter ast_note_interactivity kernel option so jupyter displays multiple statements at once
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Import all modules/libraries which will be loaded within this project

In [ ]:
import itertools
from time import time
from pprint import pprint
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV,  RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn import naive_bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import tree
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_curve, auc, classification_report, confusion_matrix
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.model_selection import cross_val_score, StratifiedKFold
##Importing hyperopt libraries
from functools import partial
from hyperopt import fmin, hp, tpe, Trials, space_eval
from hyperopt.pyll import scope

### Section 2: Data Analysis
---

##### 2.1 Data Exploration

In [ ]:
# 2.1 Data Exploration

# Get the current working directory
current_directory = os.getcwd()

# Construct the file path
file_path = os.path.join(current_directory, 'data', 'oasis_longitudinal_demographics.csv')

# Load in the data
mri = pd.read_csv(file_path)

# Get a feel for the data - observe first 5 rows
print("First 5 rows of the data:")
print(mri.head(5))

# Print out the number of columns in the dataframe (features)
num_columns = len(mri.columns)
print(f"\nThere are {num_columns} attributes within the dataset.")

# Check the columns of the dataset in prep for Data Exploration
print("\nColumns of the dataset:")
print(mri.columns)

In [ ]:
# Check the columns and their type - info prints this info about a dataframe
print("Information about the dataset:")
mri.info()

# Dimensions of the dataset
num_columns = len(mri.columns)
num_rows = len(mri.index)
print(f"\nThe dataset is made up with {num_columns} columns and {num_rows} rows.")

In [ ]:
# Let's check the distinct values in certain columns
# Common sense would suggest that Subject ID, MRI ID, MR Delay, eTIV, nWBV, and ASF features would have too many distinct values for the purpose of this test.
# Therefore let's ignore them

num_columns = ['Visit', 'Age', 'EDUC', 'SES', 'MMSE', 'CDR']

for col in num_columns:
    unique_values = mri[col].unique()
    print(f"{len(unique_values)} unique values for {col}:")
    print(unique_values)
    print("\nValue counts:")
    print(mri[col].value_counts(), "\n")

cat_columns = ['Group', 'M/F', 'Hand']
for col in cat_columns:
    unique_values = mri[col].unique()
    print(f"{len(unique_values)} unique values for {col}:")
    print(unique_values)
    print("\nValue counts:")
    print(mri[col].value_counts(), "\n")


In [ ]:
#Get a summary of the numerical attributes within our dataset - let's us see if we need to normalize/scale at a later point
for col in num_columns:
    mri[col].describe()

##### 2.2 Exploratory Data Analysis

In [ ]:
# Plotting histograms side by side for pairs of variables

# Define pairs of variables
pairs = [('Age', 'CDR'), ('SES', 'EDUC'), ('MMSE', 'M/F')]

# Plot histograms for each pair of variables
for pair in pairs:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    for i, col in enumerate(pair):
        sns.histplot(mri[col], ax=axes[i], kde=False)
        axes[i].set_title(f'Histogram of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Frequency')
    plt.tight_layout()
# Adding ; to last statement to get rid of <matplotlib...> <seaborn...> text
plt.show();

A few things can be concluded from these graphs about the dataset:
- It seems that the age of the majority of Subject IDs falls between 70 - 85 with the most frequent age being 73 years old (more than 25 participants in this study are of this age). Note this could be skewed as we have not yet discounted Subject IDs that have had more than one session. 
- More than half of the datasets' population received a CDR rating of 0 (>200 Subject IDs) while less than 10 subjects were diagnosed with a 2.0 CDR score.  
- The most frequent Socio-Economic-status score in this study was 2.0 with over > 100 subject IDs (103), followed by > 85 subject IDs graded as 1.0 on the SES scale. 
- Most of the individuals in this study undertook a significant amount of years in Education with over 2/3s of the study having at least studied 12+ years of Education. 
- The majority of Subject IDs scored highly in their MMSE examinations (> 50% scoring 29 or above).
- The percentage ratio between females and males was 57:43. 

**NOTE**:

Although i will be replacing the 'Converted' values in the group attribute, for the purpose of visual analysis, I am dropping the rows for 'Converted' patients so i can directly compare 'demented' and 'non-demented' subjects. To accomodate this, I am taking a copy of the 'mri' dataframe to ensure we dont lose values.

Let's create a function called `plot_facetgrid` which will generate FacetGrid plots based on specified variables, using predefined colors, titles, and x-axis limits. It utilizes Seaborn's FacetGrid and kdeplot functions for visualization:

In [ ]:
# Create a copy of the dataframe so we can remove the 'converted' value from the 'Group' attribute
mri_copy = mri.copy()

mri_copy =mri_copy[mri_copy.Group != "Converted"]

def plot_facetgrid(data, var):
    """
    Function to create a FacetGrid chart based on the variable var.
    
    Parameters:
        data (DataFrame): The DataFrame containing the data.
        var (str): The variable to plot.
    """
    csfont_options = { 'fontname': 'monospace',
                   'weight': 'bold',
                   'color':'b',
                   'size': 'large'
                  }
    
    plot_colors_lookup = {
        'MMSE': {'color': ['r', 'k'], 'title': 'Dementia Variation against MMSE scores', 'xlim': (10, data['MMSE'].max())},
        'SES': {'color': ['r', 'k'], 'title': 'Variation of Dementia against Socio-Economic-Status', 'xlim': (0, data['SES'].max())},
        'nWBV': {'color': ['r', 'k'], 'title': 'Variation of Dementia against Normalized Whole Brain Volume', 'xlim': (0, data['nWBV'].max())},
        'eTIV': {'color': ['r', 'k'], 'title': 'Variation of Dementia against Estimated Total Intracranial volume', 'xlim': (0, data['eTIV'].max())},
        'EDUC': {'color': ['r', 'k'], 'title': 'Variation of Dementia against Years of Education', 'xlim': (3, data['EDUC'].max())},
        'Age': {'color': ['r', 'k'], 'title': 'Dementia Variation against Age', 'xlim': (10, data['Age'].max())}
    }
    
    # Lookup plot colors, title, and xlim based on the variable var
    plot_colors = plot_colors_lookup[var]['color']
    title = plot_colors_lookup[var]['title']
    xlim = plot_colors_lookup[var]['xlim']
    
    # Create FacetGrid
    g = sns.FacetGrid(data, hue='Group', hue_kws={'color': plot_colors}, height=8)
    
    # Plot KDE plot
    g.map(sns.kdeplot, var, shade=True)
    
    # Set x-axis limit
    g.set(xlim=xlim)
    
    # Add legend
    plt.legend(loc='best')
    
    # Set title
    plt.title(title, **csfont_options)
    
    # Show plot
    plt.show()


In [ ]:
plot_facetgrid(mri_copy, 'MMSE')

From the above graph, one can see the Dementia distribution as a function of MMSE score. For high MMSE scores, in approximately 70% of cases, the subject IDs are generally considered as non-demented . Finally, as expected, for low MMSE scores (<22) all subjects receive a 'demented' diagnosis.

In [ ]:
plot_facetgrid(mri_copy, 'SES')

Interestingly, the distribution of Dementia across the study is greatest for subjects who have receive a score in the socio-economic-status scale of 3 or more. This possibly suggests that the more afluent an individual is, the greater the chance of Dementia. Subjects who generally score 3 or less are considered 'Non-Demented' (in >50% of cases for a score of 2) or 'Converted' (in ~60% of cases where subjects score 1 in the SES scale)   

In [ ]:
plot_facetgrid(mri_copy, 'nWBV')

As one would expect, subjects who exhibited a greater percentage of normalized whole brain volume, had a greater chance of receiving a 'Non-demented' diagnosis.  The lower the nWBV value, the greater the chance a patient was classified as 'demented' or 'converted'. In recent years, brain shrinkage has become a potential important marker for early changes in the brain tissue where such symptoms have been associated as early markers for ailments of Alzheimer's [https://www.webmd.com/alzheimers/news/20110413/brain-shrinkage-may-help-predict-alzheimers].

In [ ]:
plot_facetgrid(mri_copy, 'eTIV')

Interestingly, there didn't seem to be much difference between the variation of Dementia with ETIV i.e. the estimated total intracranial volume didn't vary much over time and therefore ETIV was not significantly different between any of classification groups. As referenced in the Paper "Inracranial Volume and Alzheimer Disease: Evidence agaisnt the cerebal Reserve Hypothesis" by Jenkins et al,  the Mean TIV did not differ significantly between subjects and that the only significant predictor of TIV was sex. Thus it was concluded that their findings do not suggest that a larger brain provides better 'protection' against AD [https://www.ncbi.nlm.nih.gov/pubmed/10681081]. This explains the visual cues seen in the plot above  .i.e 'Demented' and 'nondemented' subjects have relatively the same ETIV values. 

In [ ]:
plot_facetgrid(mri_copy, 'EDUC')

It seems from quickly glancing at the above, Dementia is more prevelant in subjects who had fewer years in Education. For e.g. any subject who had less than 13 years of Education had a slightly increased possibility of developing Dementia where as the chances of having Dementia-like symptoms (on first visit) decreases quite sharply for subjects who have had 17+ years in Education.

Finally, Dementia is more prevelant in subjects who fall within the 65 - 85 year group. Before 65, one could hypothesise that the subject is too young for the full blown effects of Dementia to manifest. Similarly, after 85, given that a patient could have been living with the disease for many years,certain symptoms could have had a profound effect on an individual's life. This possibly explains why the dementia (not converted) diagnosis is small after the years of 85. Obviously, Dementia forms in people at different stages, as shown by the converted plot where it gets larger in magnitude as ages increase (up to ~85 years old). This would suggest that a subject's condition has been updated from a 'non-demented' to 'demented' status in that timeframe. For the purpose of visual analysis and for reasons of brevity, I'm only discussing the direct comparisons between the 'demented' and 'non-demented' plots.

In [ ]:
plot_facetgrid(mri_copy, 'Age')

If you want to see the 'Converted' group plotted against both 'demented' & 'non-demented' groups:

In [ ]:
# Define the target variable and features
targ = 'Group'
cols = ['MMSE', 'SES', 'nWBV', 'eTIV', 'EDUC', 'Age']

# Define font options for the title
csfont_options = {'fontname': 'monospace', 'weight': 'bold', 'color': 'b', 'size': 'large'}

# Set min x-axis values for each feature
xlimits = [12, 0, 0.55, 840, 3, 50]

# Define plot colors for different groups
plot_colors = {'Converted': 'r', 'Demented': 'k', 'Non-Demented': 'b'}

# Iterate over the target variable and features
for feature, xlimit in zip(cols, xlimits):
    g = sns.FacetGrid(mri, hue=targ, height=8)
    g.map(sns.kdeplot, feature, shade=True)
    g.set(xlim=(10, mri[feature].max()))
    plt.legend(loc='best')
    plt.title('Dementia Variation against {}'.format(feature), **csfont_options)
    plt.xlim(xlimit)

plt.show();


Out of curiousity, plot the frequency gender for Demented/Non-Demented subjects:

In [ ]:
g = sns.catplot(x='M/F', 
                data=mri_copy, hue='Group', 
                kind='count', palette="muted", 
                legend=False, height=6)

g.despine(left=True)
g.set_ylabels("Participation count")
plt.legend(loc='best')
plt.show();

##### 2.2.2 Correlation

In [ ]:
def plot_corr_heatmap(data, method, heading):
    """
    Plot correlation heatmap for the given data.

    Parameters:
        data (DataFrame): The DataFrame containing the data.
        method (str): The method used for computing correlations.
        heading (str): The title for the heatmap.
    """
    # Copy mri dataset and preprocess
    mri_copy = data.drop(['Subject ID', 'M/F', 'MRI ID', 'Hand'], axis=1)
    mri_copy = pd.get_dummies(mri_copy, drop_first=True)

    # Compute correlation matrix
    corr = mri_copy.corr(method=method)

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(12, 8))

    # Draw the heatmap
    sns.heatmap(corr, annot=True, linewidths=.5, fmt='.2f', ax=ax)

    # Set title
    ax.set_title(heading)

    # Show plot
    plt.show()

    return corr

plot_corr_heatmap(mri, 'pearson', 'Correlation Heat map')

Observing the correlation coefficient (pearsons r) between each pair of attributes, I can make several assumptions:
- CDR has a strong positive correlation with both Group_Demented and Group_Nondemented. This is expected given that the CDR scale is used to gauge whether a patient has symptoms of Dementia. Given that this feature is strongly correlated with the target attribute, it would be advisable to drop this column during feature selection.
- Age and nWBV have quite a strong negative correlation. 
- EDUC has quite strong correlations with MMSE, eTIV and a v.strong negative correlation with SES
- MMSE and CDR share a strong -ive correlation , quite strong correlation with nWBV. 
- Most of the attributes share a correlation that is close to 0 thus we assume that there is little to no linear correlation

Note, these **correlations are only linear and as a result we arent measuring non-linear relationships that could exist within the dataset (if attribute x is close to 0, y goes up) **

##### 2.2.3 Scatter Matrix

In [ ]:
scatter_matrix(mri[mri.columns], figsize=(12,8));

### Section 3: Data preprocessing
---

##### 3.1 Categorical Attribute replacement

In [ ]:
# Display unique values before and after replacement
for col in ['Group', 'M/F']:
    print(f"Before replacement {col} has the following values: {mri[col].unique()}")

# Create a mapping dictionary for replacement
mapping = {'F': 0, 'M': 1, 'Nondemented': 0, 'Demented': 1, 'Converted': 1}

# Replace categorical values with numerical values
mri.replace({'Group': mapping, 'M/F': mapping}, inplace=True)

# Display unique values after replacement
for col in ['Group', 'M/F']:
    print(f"After replacement {col} has the following numerical values: {mri[col].unique()}")

In [ ]:
## Using LabelEncoder from scikit learn to encode categorical features to numerical.
obj_mri = mri.select_dtypes(include=['object'])

# Converting categorical Data 
for col in obj_mri.columns:
    le = LabelEncoder()
    le.fit(mri[col])
    list(le.classes_)
    mri[col] = le.transform(mri[col])

In [ ]:
# Check that all columns are now numerical
mri.info()

##### 3.1.2 Outlier detection

In [ ]:
def outlier_detection(cols):
    """
    Detect outliers in a list of values using z-score method.
    
    Parameters:
        cols (list): A list of numerical values.
    
    Returns:
        outliers (tuple): A tuple containing the indices of outliers.
    """
    mean_col = np.mean(cols)
    std_col = np.std(cols)
    threshold = 3
    z_scores = [(col - mean_col) / std_col for col in cols]
    outliers = np.where(np.abs(z_scores) > threshold)
    return outliers

In [ ]:
cols = ["EDUC", "SES", "MMSE", "CDR", "eTIV", "Visit", "eTIV", "nWBV", "ASF"]

for col in cols:
    outliers_indices = outlier_detection(mri[col])
    if len(outliers_indices[0]) > 0:
        print(f"Outliers for {col} are given below:")
        for i in outliers_indices[0]:
            outlier_value = mri[col].iloc[i]
            print(f"Index: {i}, Value: {outlier_value}")
        print()


The outliers for MMSE, CDR and Visit have actual 'meaning' ... i.e. MMSE scores of 16,15,7,4 etc are used to determine the severity of Dementia in a subject (same applies to CDR). Thus, I believe it wouldn't be suitable to drop these outliers from the dataset

##### 3.1.3 Imputation - dealing with missing values

In [ ]:
#Check which columns contain null values
nulls = mri.columns[mri.isnull().any()]

In [ ]:
#Check how many nulls in each column
mri[nulls].isnull().sum()

In [ ]:
# Imputation - only works on numerical attributes, hence why we encoded the categorical text features in 3.1
# Impute missing values with the median value of the attribute
imputer = SimpleImputer(strategy="median")

# Final check to make sure all columns are numerical before proceeding with imputation
for column in mri.columns:
    if mri[column].dtype == 'object':
        print(f"{column} of type object found!")
    else:
        print(f"{column} is a numerical attribute - proceed with imputation")

# Fitting imputer instance
imputer.fit(mri)

# Check if the median computed by the imputer matches the median of the original data
print("Medians match:", (imputer.statistics_ == mri.median().values).all())

# Transforming mri data to replace null values via imputation
X = imputer.transform(mri)

# Constructing dataframe from transformed values
mri = pd.DataFrame(X, columns=mri.columns)


In [ ]:
#Check that nulls have been replaced
mri[nulls].isnull().sum()

##### 3.1.4 Standardizing dataframe attributes

In [ ]:
cols = ['Age', 'M/F', 'EDUC', 'SES', 'MMSE', 'eTIV', 'nWBV', 'ASF']

max_values = mri[cols].max()
min_values = mri[cols].min()

print(f"max value in mri dataset for {cols} before scaling:\n{max_values}")
print(f"min value in mri dataset for {cols} before scaling:\n{min_values}")

In [ ]:
scaler = StandardScaler().fit(mri)
mri[cols]=scaler.fit_transform(mri[cols])

In [ ]:
cols = ['Age', 'M/F', 'EDUC', 'SES', 'MMSE', 'eTIV', 'nWBV', 'ASF']

max_values = mri[cols].max()
min_values = mri[cols].min()

print(f"max value in mri dataset for {cols} before scaling:\n{max_values}")
print(f"min value in mri dataset for {cols} before scaling:\n{min_values}")

**Note**: We only standardized certain attributes, as some of these columns will be dropped during feature selection in our next preprocessing step

##### 3.1.5 Dropping features

In [ ]:
# Drop unneccessary features and also the highly correlated feature CDR from the dataframe
mri = mri.drop(['Subject ID', 'Visit', 'MRI ID', 'Hand', 'MR Delay', 'CDR'], axis=1)

In [ ]:
mri.head(10)

##### 3.1.6 Splitting data into training / test sets

In [ ]:
seed = np.random.seed(42)
y =  mri['Group']   #Target 
X = mri.drop(['Group'], axis=1) # Features

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Create copies of our train/test sets for benchmark testing
BM_X_train = X_train.copy()
BM_X_test = X_test.copy()
BM_y_train = y_train.copy()
BM_y_test = y_test.copy()

#Create further copies incase of further optimization techniques
Copy_X_train = X_train.copy()
Copy_X_test = X_test.copy()
Copy_y_train = y_train.copy()
Copy_y_test = y_test.copy()


In [ ]:
print("X_train set is made up of {} rows and {} columns\n".format(len(X_train.index), len(X_train.columns)))
print("X_test set is made up of {} rows and {} columns\n".format(len(X_test.index), len(X_test.columns)))
print("y_train is made up of {} values\n".format(len(y_train.index)))
print("y_test is made up of {} values\n".format(len(y_test.index)))

##### 3.2 Data Implementation

---
 **Benchmark Statistics**

In [ ]:
## As per the evaluators suggestion, look into using xgboost  -> popular boosting model -> appropiate here
#setting random state as 42 for reproducability

# Define the models
models = [ 
    LogisticRegression(random_state=42),
    SVC(random_state=42),
    DecisionTreeClassifier(random_state=42),
    AdaBoostClassifier(random_state=42),
    RandomForestClassifier(random_state=42),
    XGBClassifier(random_state=42),
    GradientBoostingClassifier(random_state=42),
    GaussianNB()
]

# Define the corresponding classifier names
classifiers = [
    'Logistic Regression',
    'SVC',
    'Decision Tree Classifier',
    'AdaBoost Classifier',
    'Random Forest Classifier',
    'XGBClassifier',
    'GradientBoostingClassifier',
    'Naive Bayes'
]

# Define the columns for the benchmark dataframe
ml_columns = ['ML Algo name', 'Model Parameters', 'training AUC accuracy', 'test recall score', 'test AUC score']

# Create an empty dataframe to store benchmark results
benchmark = pd.DataFrame(columns=ml_columns)

# Perform cross-validation and evaluation for each model
for model, clf_name in zip(models, classifiers):
    benchmark.loc[len(benchmark)] = [clf_name, str(model.get_params()), np.nan, np.nan, np.nan]
    cv_score = cross_val_score(model, BM_X_train, BM_y_train, cv=5, scoring='roc_auc') 
    benchmark.loc[benchmark['ML Algo name'] == clf_name, 'training AUC accuracy'] = np.mean(cv_score)
    mdl = model.fit(BM_X_train, BM_y_train)
    test_acc = mdl.score(BM_X_test, BM_y_test)
    fpr, tpr, thresholds = roc_curve(BM_y_test, model.predict(BM_X_test))
    test_recall = recall_score(BM_y_test, model.predict(BM_X_test))
    test_auc = auc(fpr, tpr)
    benchmark.loc[benchmark['ML Algo name'] == clf_name, 'test recall score'] = test_recall
    benchmark.loc[benchmark['ML Algo name'] == clf_name, 'test AUC score'] = test_auc 

# Sort benchmark table by test AUC score
benchmark.sort_values(by='test AUC score', ascending=False, inplace=True)

# Plot the benchmark results
sns.barplot(x='test AUC score', y='ML Algo name', data=benchmark)

# Set the width of the model parameters column
benchmark.style.set_properties(subset=['Model Parameters'], **{'width': '300px'})


Plotting the roc_curves and confusion metrics for evaluation purposes ...

In [ ]:
total_fpr = {}
total_tpr = {}

def roc_curves(model, classifier):
    ML_name = model.__class__.__name__
    fpr, tpr, thresholds = roc_curve(BM_y_test, model.predict(BM_X_test))
    test_auc = auc(fpr, tpr)
    total_fpr[ML_name] = fpr
    total_tpr[ML_name] = tpr
    plt.plot(fpr, tpr, color='darkorange', linewidth=2, label='(Area = %0.2f)' % test_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.title('ROC Curve for {}'.format(classifier))
    plt.show()

def confusion_matrix_plot(model, classifier):
    cm = confusion_matrix(y_test, model.predict(BM_X_test))
    plt.imshow(cm, interpolation='nearest', cmap=plt.get_cmap('Blues'))
    labels = ['Nondemented', 'Demented']
    plt.title('Confusion Matrix for {}'.format(classifier))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels)
    plt.yticks(tick_marks, labels)
    s = [['TN', 'FP'], ['FN', 'TP']]
    for i, j in itertools.product(range(2), range(2)):
        plt.text(j, i, str(s[i][j]) + " = " + str(cm[i][j]))
    plt.show()

for model, classifier in zip(models, classifiers):
    roc_curves(model, classifier)
    confusion_matrix_plot(model, classifier)

Plotting all of the ROC curves on one graph ...

In [ ]:
colors = {
    'LogisticRegression': 'red',
    'SVC': 'green',
    'DecisionTreeClassifier': 'blue',
    'AdaBoostClassifier': 'yellow',
    'RandomForestClassifier': 'cyan',
    'XGBClassifier': 'magenta',
    'GradientBoostingClassifier': 'black',
    'GaussianNB': 'white'
}

plt.figure(figsize=(20, 12))
for model_name in total_fpr.keys():
    plt.plot(total_fpr[model_name], total_tpr[model_name], color=colors[model_name], lw=1, label=model_name)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.title('Comparison of ROC curves for all classifiers')
plt.plot([0, 1], [0, 1], color='darkorange', lw=2, linestyle='--')
plt.legend()
plt.show();


##### 3.2.2 Implementation
---
**Optimization techniques**

I made the mistake of trying to perform GridSearchCV for all classifiers. As you can see in the markdown below, the hyperparameter tuning of the Random Forests Classifier and the decision tree classifier took approximately 135746 & 37069 seconds or in more explicit scaling, upwards  of 38 / 10 hours. 

Thus, I will **only be performing GridSearchCV** for LogisticRegression and SVM.

The **remaining classifiers will be tuned using RandomizedSearchCV and hyperopt**


### GridSearch/RandomizedSearchCV

The below function 'grid' uses GridSearchCV to find optimal hyperparamters for the Logistic Regression and SVM Models. Due to the vast parameter grids for the remaining Models, it was advisable to tune their hyperparameters via RandomizedSearchCV. 

Also, the 3 benchmark models (SVM, RandomForestClassifier,XGBoost) which i wanted to pay particular attention to,are also fitted for the evaluation metric 'accuracy'. Reasoning behind this - I wanted to collect the classification error on training/test sets ... the classification error being equal to (1 - accuracy_score)... to test the robustness of each model.

In [ ]:


#Create empty final results dataframe for optimization metrics
refinement_cols = ['ML Model','Method','fitting time', 'optimal hyperparameters', 'best training AUC score', 'best estimator','test recall score', 'test AUC score']
finale = pd.DataFrame(columns = refinement_cols)

#Create empty robustness table to compare training and testing errors
robust_cols = ['ML Model', 'Training Classification Error', 'Test Classification_Error']
robustment_test = pd.DataFrame(columns = robust_cols)

#Storing training times
training_time = []
#Create two empty dictionaries so that the FPR and TPR can be captured for each model. This will allow us to plot all the ROC curves on one graph
total_fpr_opt = {} 
total_tpr_opt = {}



#Define classifiers for tuning - set random_state for reproducible results
classifiers = [ LogisticRegression(random_state=42),
                SVC(random_state=42),
                AdaBoostClassifier(random_state=42),
                DecisionTreeClassifier(random_state=42),
                RandomForestClassifier(random_state=42),
                GradientBoostingClassifier(random_state=42),
                XGBClassifier(random_state=42) ]



lr_parameters = { 
                  'C':[0.001, 0.01, 0.1, 1, 10,100,1000, 10000], 
                  'max_iter':list(range(2,100))}


svm_parameters = {
                  'kernel': ['rbf', 'poly', 'sigmoid'],
                  'C':[0.00001,0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 30, 40, 50 ,60 , 80, 100],
                  'gamma': [0.00001,0.0001, 0.001, 0.01, 0.1, 1, 10] }

dt_parameters = { #'max_leaf_nodes': list(range(2,100)),
                  'max_leaf_nodes': list(range(2,20)),
                  'splitter':['random', 'best'],
                  'criterion': ['gini', 'entropy'],
                  'max_depth': list(range(2,20)),
                  'min_samples_split': list(range(2,20))}

rf_parameters = { 
                  'n_estimators': [10,30,50,100,150,200,250,300,350,400,500,600,700,800,900,1000],
                  'max_leaf_nodes': list(range(2,30)),
                  'min_samples_leaf':[1,2,3], 'min_samples_split':[3,4,5,6,7]}

gb_parameters = {
                  'learning_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                  'min_samples_leaf': list(range(1,20)),
                  'max_depth': list(range(2,20)),
                  'n_estimators': range(1,200)}

xgboost_parameters = { 
                  'silent': [True],
                  'max_depth': [6, 10, 15, 20],
                  'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
                  'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                  'colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                  'colsample_bylevel': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                  'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
                  'gamma': [0, 0.25, 0.5, 1.0],
                  'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
                  'n_estimators': [50,100,120,150]}


ada_parameters = { 
                   'n_estimators': [50,100,200,250,300,400,500,600,700,800,900,1000],
                   'learning_rate': [ 0.1,0.15, 0.2,0.25, 0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] }

parameters = [lr_parameters, 
              svm_parameters,
              ada_parameters, 
              dt_parameters,
              rf_parameters, 
              gb_parameters, 
              xgboost_parameters]  
                    



In [ ]:
os.environ["PARALLEL_JOBS"] = "16"

In [ ]:
def get_n_jobs():
    parallel_jobs = os.getenv('PARALLEL_JOBS')
    if parallel_jobs is not None:
        try:
            n_jobs = int(parallel_jobs)
            return n_jobs
        except ValueError:
            print("Error: PARALLEL_JOBS is not an integer. Using default value (None).")
            return None
    else:
        return None

parallel_jobs = get_n_jobs()

def print_summary(ML_name, mdl, method, fitting_time, best_params, confusion_matrix, complete_time,parallel_jobs,feature_importances=False):
    print("--" * 50)
    print("Finding the best hyperparameters for metric [roc_auc] for {}\n".format(ML_name))
    print("Performing {} for {}\n".format(method, ML_name))
    print("Beginning to fit {}\n".format(ML_name))
    print("Optimal hyperparameters:", best_params)
    print("Confusion matrix:")
    confusion_matrix_plot(confusion_matrix, ML_name)
    if feature_importances:
        print_feature_importances(ML_name, mdl)
    if parallel_jobs == None:
        parallel_jobs = 1
    print(f"Hyperparameter tuning complete. Used {parallel_jobs} parallel jobs... Took {complete_time:.2f} seconds.\n")

def print_feature_importances(ML_name, mdl):
    print("=="*16,"feature_importances","=="*16)
    for name, importance in zip(X_train.columns, mdl.feature_importances_):
        print(name, "==", importance)
    importances = mdl.feature_importances_
    indices = np.argsort(importances)
    features = X_train.columns
    plt.title('Feature Importances')
    plt.barh(range(len(indices)), importances[indices], color='b', align='center')
    plt.yticks(range(len(indices)), [features[i] for i in indices])
    plt.xlabel('Relative Importances')
    plt.show();

def confusion_matrix_plot(confusion_matrix, classifier):
    plt.imshow(confusion_matrix, interpolation='nearest', cmap=plt.get_cmap('Blues'))
    labels = ['Nondemented', 'Demented']
    plt.title('Confusion Matrix for {}'.format(classifier))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels)
    plt.yticks(tick_marks, labels)
    s = [['TN', 'FP'], ['FN', 'TP']]
    for i, j in itertools.product(range(2), range(2)):
        plt.text(j, i, str(s[i][j]) + " = " + str(confusion_matrix[i][j]))
    plt.show()

def grid(classifier, parameters,jobs):
    ML_name = classifier.__class__.__name__
    start_time = time()
    models = ['DecisionTreeClassifier', 'RandomForestClassifier', 'XGBClassifier', 'GradientBoostingClassifier', 'AdaBoostClassifier']
    if ML_name in models:
        method = "Randomized Grid Search"
        grid = RandomizedSearchCV(estimator=classifier, param_distributions=parameters, cv=10, n_iter=100, scoring="roc_auc", random_state=42,n_jobs=jobs,verbose=0)
    else:
        method = "Grid Search"
        grid = GridSearchCV(estimator=classifier, param_grid=parameters, cv=10, scoring="roc_auc",n_jobs=jobs,verbose=0)
    grid.fit(X_train, y_train)
    fitting_time = time() - start_time
    best_params = grid.best_params_
    mdl = grid.best_estimator_.fit(X_train, y_train)
    pred = mdl.predict(X_test)
    test_acc = accuracy_score(y_test, pred)
    test_recall = recall_score(y_test, pred, pos_label=1)
    fpr,tpr,thresholds = roc_curve(y_test, pred, pos_label=1)
    test_auc = auc(fpr, tpr)
    total_fpr_opt[ML_name] = fpr  #Dit key
    total_tpr_opt[ML_name] = tpr #Dict key
    conf_matrix = confusion_matrix(y_test, pred)
    complete_time = time() - start_time
    # Feature importances if applicable
    feature_importances = None
    if ML_name in models:
        feature_importances = True

    # Inserting data into finale DataFrame
    finale.loc[index, 'ML Model'] = ML_name
    finale.loc[index, 'Method'] = method
    finale.loc[index, 'fitting time'] = fitting_time
    finale.loc[index, 'optimal hyperparameters'] = str(best_params)
    finale.loc[index, 'best training AUC score'] = grid.best_score_
    finale.loc[index, 'best estimator'] = str(mdl)
    # Inserting test recall score and test AUC score into finale DataFrame
    finale.loc[index, 'test recall score'] = test_recall
    finale.loc[index, 'test AUC score'] = test_auc
    print_summary(ML_name, mdl, method, fitting_time, best_params, conf_matrix, complete_time,jobs,feature_importances=feature_importances)
    # Robustness check for certain models
    robust_models = ['SVC', 'RandomForestClassifier', 'XGBClassifier']
    if ML_name in robust_models:
        print("------------- Calculating {} classification error for Robustness checks  -------------\n".format(ML_name))
        if ML_name in models:
            grid = RandomizedSearchCV(estimator=classifier, param_distributions=parameters, cv=10, n_iter=100, scoring="accuracy")
            robustment_test.loc[index, 'ML Model'] = ML_name
            grid.fit(X_train, y_train)
        else:
            grid = GridSearchCV(estimator=classifier, param_grid=parameters, cv=10, scoring="accuracy")
            robustment_test.loc[index, 'ML Model'] = ML_name
            grid.fit(X_train, y_train)
        training_error = 1 - (grid.best_score_)
        robustment_test.loc[index, 'Training Classification Error'] = training_error
        mdl = grid.best_estimator_.fit(X_train, y_train)
        pred = mdl.predict(X_test)
        test_error = 1 - (accuracy_score(y_test, pred))
        robustment_test.loc[index, 'Test Classification_Error'] = test_error
        print("Training classification error: ", training_error)
        print("Testing classification error: ", test_error)
        print("\n Robustness checks complete!")


index = 0
for classifier, parameter in zip(classifiers, parameters):
    grid(classifier, parameter,parallel_jobs)
    index += 1


In [ ]:
finale.head(10)
finale[['ML Model','test AUC score']]

In [ ]:
#Check classification scores for the 3 main benchmark models
robustment_test

Plotting the ROC curve for each classifier ...

In [ ]:
##Setting the colour of the ROC curves for each model
colors = { 'LogisticRegression':'red',
            'SVC':'green',
            'DecisionTreeClassifier':'blue',
            'AdaBoostClassifier':'yellow',
            'RandomForestClassifier':'cyan',
            'XGBClassifier':'magenta', 
            'GradientBoostingClassifier': 'black'
        }    

#total_fpr.keys() returns the models name as the key
plt.figure(figsize=(20,12))
for i in total_fpr_opt.keys():
    colors = {'LogisticRegression':'red', 'SVC':'green', 'DecisionTreeClassifier':'blue', 'AdaBoostClassifier':'yellow','RandomForestClassifier':'cyan', 'XGBClassifier':'magenta', 'GradientBoostingClassifier': 'black', 'GaussianNB': 'white'}    
    plt.plot(total_fpr_opt[i],total_tpr_opt[i],colors[i], lw=1, label=i)
plt.xlim([0,1])
plt.ylim([0,1])
plt.title('Comparison of ROC curves for all classifiers')
plt.plot([0, 1], [0, 1], color='darkorange', lw=2, linestyle='--')
plt.legend();

### Further Hyperparameter refinement using hyperopt

In [ ]:
def Performance(model, y, X, fitting_time, best_params):
    predictions = model.predict(X)
    recall = recall_score(y, predictions, pos_label=1)
    fpr, tpr, _ = roc_curve(y, predictions)
    auc_score = auc(fpr, tpr)
    print('Recall score: {:.4f}'.format(recall))
    print('AUC score: {:.4f}'.format(auc_score))
    
    # Update finale table
    global index
    finale.loc[index, 'ML Model'] = model.__class__.__name__
    finale.loc[index, 'Method'] = 'Hyperopt tuning'
    finale.loc[index, 'fitting time'] = str(fitting_time)
    finale.loc[index, 'optimal hyperparameters'] = str(best_params)
    finale.loc[index, 'best training AUC score'] = auc_score
    finale.loc[index, 'best estimator'] = str(model)
    finale.loc[index, 'test recall score'] = recall
    finale.loc[index, 'test AUC score'] = auc_score
    
    # Increment index for the next entry
    index += 1

---
<font color=purple>*XGBoost hyperopt*</font>

---

In [ ]:
def objective(params):
    params = {
        'n_estimators': params['n_estimators'],
        'learning_rate': "{:.3f}".format(params['learning_rate']),
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'min_child_weight': "{:.3f}".format(params['min_child_weight']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'random_state': params['random_state']
    }
    
    #clf = XGBClassifier(n_jobs=4,**params)
    clf = XGBClassifier(**params)
    skf = StratifiedKFold(n_splits=10, shuffle=True,random_state=42)
    score = cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=StratifiedKFold()).mean()
    print("roc_auc {:.3f} params {}".format(score, params))
    training_scores.append(score)
    return score

space = {
    'n_estimators': scope.int(hp.uniform('n_estimators',100, 2000)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
    'max_depth': scope.int(hp.quniform('max_depth', 2, 14, 1)),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'min_child_weight': hp.uniform('min_child_weight',0.1, 0.6),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
    'random_state': 42 
}

#Capture training scores & fitting times as well
training_scores = []
start_time = time()
best = fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=10)
best_params = space_eval(space, best)
fitting_time = time() - start_time
print("\nHyperopt completed in {} seconds\n".format(fitting_time))
print("The average training score was {} ". format(sum(training_scores)/len(training_scores)))



Using the best parameters computed using Hypteropt to calculate a **Recall** and **AUC** score:

In [ ]:
# Call Performance function after hyperparameter tuning
mdl = XGBClassifier(**best_params)
mdl.fit(X_train, y_train)
Performance(mdl, y_test, X_test, fitting_time, best_params)


---
<font color=Purple>*RandomForest hyperopt*</color>

---

In [ ]:
def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'n_estimators':int(params['n_estimators']),
        'random_state': params['random_state']
    }
    
    #clf = RandomForestClassifier(n_jobs=4,**params)
    clf = RandomForestClassifier(**params)
    score = cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=StratifiedKFold()).mean()
    print("roc_auc {:.3f} params {}".format(score, params))
    training_scores.append(score)
    return score

space = {
    'max_depth': scope.int(hp.quniform('max_depth', 10,30, 1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 100,1500,25)),
    'random_state': 42 }

#Capture training scores & fitting times as well
training_scores = []
start_time = time()
best = fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=10)
best_params = space_eval(space, best)
fitting_time = time() - start_time
print("\nHyperopt completed in {} seconds\n".format(fitting_time))
print("The average training score was {} ". format(sum(training_scores)/len(training_scores)))

In [ ]:
# Call Performance function after hyperparameter tuning
mdl = RandomForestClassifier(**best_params)
mdl.fit(X_train, y_train)
Performance(mdl, y_test, X_test, fitting_time, best_params)

---
<font color=Purple>*SVC hyperopt*</color>

---

In [ ]:
def objective(params):
    params = {
        'C': params['C'],
        'gamma':params['gamma'],
        'kernel': params['kernel'],
        'random_state': params['random_state']
    }
    
    clf = SVC(**params)
    score = cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=StratifiedKFold()).mean()
    print("roc_auc {:.3f} params {}".format(score, params))
    training_scores.append(score)
    return score

space = {
    'C': hp.uniform('C', 10,11),
    'gamma': hp.quniform('gamma', 0.35,0.42,0.01),    
    'kernel': 'rbf', 
    'random_state': 42 }

#Capture training scores & fitting times as well
training_scores = []
start_time = time()
best = fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=10)
best_params = space_eval(space, best)
fitting_time = time() - start_time
print("\nHyperopt completed in {} seconds\n".format(fitting_time))
print("The average training score was {} ". format(sum(training_scores)/len(training_scores)))

In [ ]:
# Call Performance function after hyperparameter tuning
mdl = SVC(**best_params)
mdl.fit(X_train, y_train)
Performance(mdl, y_test, X_test, fitting_time, best_params)

### Section 4: Results
---

##### 4.1 Model Evaluation and Validation

Going to compare the 2 dataframes -- finale & benchmark -- to show the differences due to parameter tuning ...

In [ ]:
def process_results(finale, benchmark):
    # Rename column in benchmark DataFrame
    benchmark.rename(columns={'ML Algo name': 'ML Model'}, inplace=True)
    
    # Extract relevant columns from finale DataFrame and sort by ML Model
    output = finale[['ML Model', 'Method', 'test AUC score']].sort_values('ML Model')
    
    # Set Method column to 'untuned' in benchmark DataFrame
    benchmark['Method'] = 'untuned'
    
    # Extract relevant columns from benchmark DataFrame
    benchmark = benchmark[['ML Model', 'Method', 'test AUC score']]
    
    # Concatenate output and benchmark DataFrames, sort by ML Model, and set index
    df = pd.concat([output, benchmark]).sort_values('ML Model')
    df.set_index('ML Model', inplace=True)
    
    return df


In [ ]:
output = process_results(finale,benchmark)
output

In [ ]:
# Extract the 'untuned' test AUC score for each ML Model and store it in a dictionary
untuned_dict = {}
for model, group in output[output['Method'] == 'untuned'].groupby('ML Model'):
    untuned_dict[model] = group.iloc[0]['test AUC score']

# List of methods to check against
methods_to_check = ['Randomized Grid Search', 'Hyperopt tuning', 'Grid Search']

# Print out the test AUC scores for each ML model with Randomized Grid Search method that outperformed the untuned method
for model, group in output.groupby('ML Model'):
    if any(method in group['Method'].values for method in methods_to_check):
        for method in methods_to_check:
            if method in group['Method'].values:
                method_score = group[group['Method'] == method]['test AUC score'].values[0]
                if model in untuned_dict and method_score > untuned_dict[model]:
                    print(f"The following model and optimisation method outperformed its benchmark value of {untuned_dict[model]}")
                    print(f"ML Model: {model}")
                    print(f"Method: {method}")
                    print(f"Test AUC score: {method_score}")
                    print()


### Section 5: Conclusion
---

##### 5.1 Free-form visualization of SVM Decision Boundaries

In [ ]:
#Define a meshgrid function that will plot the decision surface
def make_meshgrid(x, y, h=.02):
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    return xx, yy

def plot_contours(ax, clf, xx, yy, **params):
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

#Select 2 features randomly to compare - SES / EDUC in this case
#Convert to array
X = np.array(X_test)
X = X[:, [2,3]]

#ax1 --> benchmark model
#ax2 --> optimised model
fig, (ax1, ax2) = plt.subplots(1,2, sharey=True, figsize=(20,10))
title = ('Decision surface for Benchmark model')
X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)
mdl = SVC(C=1, gamma='auto', kernel='rbf', random_state=42).fit(X, y_test)
plot_contours(ax1, mdl, xx, yy, cmap=plt.cm.coolwarm, alpha=0.7)
ax1.scatter(X0, X1, c=y_test, cmap=plt.cm.coolwarm, s=20, marker='x', edgecolors='k')
ax1.set_xlabel(X_test.columns[2])
ax1.set_ylabel(X_test.columns[3])
ax1.set_xticks(())
ax1.set_yticks(())
ax1.legend()
ax1.set_title(title);


title = ('Decision surface for Optimized model')
mdl = SVC(C=10.94, gamma=0.36, kernel='rbf', random_state=42).fit(X, y_test)
plot_contours(ax2, mdl, xx, yy, cmap=plt.cm.coolwarm, alpha=0.7)
ax2.scatter(X0, X1, c=y_test, cmap=plt.cm.coolwarm, s=25, marker='x', edgecolors='k')
ax2.set_xlabel(X_test.columns[2])
ax2.set_ylabel(X_test.columns[3])
ax2.set_xticks(())
ax2.set_yticks(())
ax2.set_title(title);